# BigQuery - BQML

This notebook uses BigQuery ML to train a model to predict the value of handwritten digits in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then evaluated and used for prediction using BigQuery directly.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-02.png">

---
## Setup

Parameters:

In [1]:
PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'

---
## Train Model

Use BigQuery ML to train multiclass logistic regression model:

In [2]:
%%bigquery
CREATE OR REPLACE MODEL `digits.digits_lr`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['target']
  ) AS
SELECT * EXCEPT(splits,target_OE)
FROM `digits.digits_prepped`
WHERE splits = 'TRAIN'

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1518.94query/s]                        


""


Review the iterations from training:

In [3]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `digits.digits_lr`)
ORDER BY iteration

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.91rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.194599,0.194156,0.2,5800
1,0,1,0.141429,0.140933,0.4,7190
2,0,2,0.087000,0.088094,0.8,6724
3,0,3,0.051789,0.055179,1.6,7187
4,0,4,0.032691,0.036865,3.2,6832
5,0,5,0.021979,0.026074,6.4,6575
6,0,6,0.015985,0.021359,12.8,5672
7,0,7,0.014307,0.019447,3.2,5405
8,0,8,0.013033,0.018772,6.4,5578
9,0,9,0.011138,0.017407,12.8,5657


---
## Evaluate Model

Review the model evaluation statistics on the Test/Train splits:

In [4]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TEST'))

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.86rows/s]


,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TRAIN,0.977487,0.977351,0.976939,0.977346,0.121603,0.999684
1,TEST,0.965547,0.967512,0.967213,0.966155,0.147033,0.999647


Review the confusion matrix for each split:

In [5]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.52rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,129,0,0,0,1,0,0,0,0,0
1,1,0,145,0,0,0,0,0,0,2,1
2,2,0,0,130,0,0,0,0,0,0,0
3,3,0,0,0,138,0,1,0,2,2,0
4,4,0,0,0,0,147,0,0,1,1,1
5,5,0,1,0,0,0,138,1,0,0,2
6,6,0,1,0,0,0,0,148,0,0,0
7,7,0,0,0,0,0,0,0,144,0,1
8,8,0,5,1,0,0,2,0,0,140,1
9,9,0,1,0,1,0,3,0,0,1,139


In [6]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TEST')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.28rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,47,0,0,0,0,0,1,0,0,0
1,1,0,33,0,0,0,0,0,0,0,1
2,2,0,2,45,0,0,0,0,0,0,0
3,3,0,0,1,38,0,0,0,1,0,0
4,4,0,0,0,0,30,0,0,0,1,0
5,5,0,0,0,0,0,38,0,0,0,2
6,6,1,0,0,0,0,0,31,0,0,0
7,7,0,0,0,0,0,0,0,34,0,0
8,8,0,0,0,0,0,0,0,1,24,0
9,9,0,0,0,0,0,0,0,0,1,34


---
## Predictions

Create a pandas dataframe with predictions for the test data in the table:

In [7]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits='TEST')
  )

Downloading: 100%|██████████| 366/366 [00:01<00:00, 272.94rows/s]


Review columns from the predictions - note that the query added columns with prefix `predicted_`

In [8]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'p0', 'p1', 'p2', 'p3',
       'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14',
       'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24',
       'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34',
       'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44',
       'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54',
       'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target',
       'target_OE', 'SPLITS'],
      dtype='object')

Print the first few rows for the columns related to the actual and predicted values:

In [9]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'SPLITS']].head()

,target,predicted_target,predicted_target_probs,SPLITS
0,1,1,"[{'label': 1, 'prob': 0.7388892193335673}, {'l...",TEST
1,5,5,"[{'label': 5, 'prob': 0.9750893661559832}, {'l...",TEST
2,3,3,"[{'label': 3, 'prob': 0.9333040289763361}, {'l...",TEST
3,6,6,"[{'label': 6, 'prob': 0.999307101847486}, {'la...",TEST
4,0,0,"[{'label': 0, 'prob': 0.9937197012594592}, {'l...",TEST


Notice the nested dictionary for predicted probabilities.  In BigQuery this is a Record type structure with nested fields for `label` and `prop`.  This is returned to the pandas dataframe as a nested dictionary.

The following code sorts the dictionary for the first record by `prop`:

In [10]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 7.690761354429482e-05},
  {'label': 1, 'prob': 0.7388892193335673},
  {'label': 2, 'prob': 0.11289917246375876},
  {'label': 3, 'prob': 0.12968869984335488},
  {'label': 4, 'prob': 0.005236890814542776},
  {'label': 5, 'prob': 0.00010281596286876272},
  {'label': 6, 'prob': 0.00036189945191721947},
  {'label': 7, 'prob': 0.00797803522570195},
  {'label': 8, 'prob': 0.0036886802262278903},
  {'label': 9, 'prob': 0.001077679064516254}]]

---
## Remove Resources
- delete model `<PROJECT_ID>.digits.digits_lr`

In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
# NOTE: This is an input for Notebook 03 - BigQuery - BQML Online Predictions 
bq.delete_model(PROJECT_ID+'.digits.digits_lr')